In [2]:
import torch
torch.cuda.is_available()

False

In [3]:
import pyterrier as pt
import numpy as np
import pandas as pd

from tqdm import tqdm
import zipfile
import glob
import ir_datasets
if not pt.started():
  pt.init()

from pyterrier_t5 import MonoT5ReRanker

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [4]:
dataset_nyt = ir_datasets.load("nyt")
dataset_wapo = ir_datasets.load("wapo/v2")

In [5]:
index_ref_nyt = pt.IndexRef.of("/app/indices/nyt/")
index_ref_wapo = pt.IndexRef.of("/app/indices/wapo/")

In [5]:
monoT5 = MonoT5ReRanker(text_field="body", batch_size=100, verbose=True)

bm25 = pt.BatchRetrieve(index_ref_wapo , wmodel='BM25', num_results=200)
mono_pipeline = bm25 >> pt.text.get_text(index_ref_wapo, "body") >> monoT5

mono_pipeline_500 = pt.BatchRetrieve(index_ref_wapo , wmodel='BM25', num_results=500) >>  pt.text.get_text(index_ref_wapo, "body") >> monoT5
mono_pipeline_50 = pt.BatchRetrieve(index_ref_wapo , wmodel='BM25', num_results=50) >>  pt.text.get_text(index_ref_wapo, "body") >> monoT5

/usr/local/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:217: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes f

In [6]:
ds1_nyt = pt.get_dataset('irds:nyt/trec-core-2017')
ds1_wapo = pt.get_dataset('irds:wapo/v2/trec-core-2018')

In [7]:
ds1_nyt.get_qrels()

,qid,docno,label,iteration
0,307,1001536,1,0
1,307,1002887,1,0
2,307,1005682,0,0
3,307,1007340,1,0
4,307,101295,1,0
...,...,...,...,...
30025,690,991749,0,0
30026,690,993504,0,0
30027,690,994345,0,0
30028,690,995425,0,0


In [8]:
for i, row in ds1_nyt.get_topics('title').iterrows():
    query = row['query']

    with open(f"/workspace/data/nyt/topics/topic.{i+1}", "w") as f:
        f.write(query)

[INFO] [starting] https://trec.nist.gov/data/core/core_nist.txt
[INFO] [finished] https://trec.nist.gov/data/core/core_nist.txt: [00:00] [24.4kB] [229kB/s]


In [10]:
nyt_topics = ds1_nyt.get_topics()

There are multiple query fields available: ('title', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.


In [12]:
nyt_topics.to_pickle("nyt_topics.pkl")

In [13]:
ds1_wapo.get_topics().to_pickle("wapo_topics.pkl")

There are multiple query fields available: ('title', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.


In [11]:
type(nyt_topics)

pandas.core.frame.DataFrame

In [1]:
ds1_nyt.get_topics()

NameError: name 'ds1_nyt' is not defined

In [11]:
title_queries = ""
for i, row in ds1_nyt.get_topics('title').iterrows():
    i_id = i+1
    query = row['query']

    line = f"1,1,{row['qid']},{query}\n"
    title_queries += line

with open(f"/workspace/data/nyt/title_queries", "w") as f:
    f.write(title_queries)

In [9]:
full_topics = ""
for i, row in ds1_nyt.get_topics().iterrows():
    title = row['title']
    description = row['description']
    narrative = row['narrative']

    line = f"1,1,{row['qid']},{title},{description},{narrative}\n"
    full_topics += line

with open(f"/workspace/data/nyt/full_topics", "w") as f:
    f.write(full_topics)

There are multiple query fields available: ('title', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.


In [14]:
#qrels
wapo_qrel_path = "/workspace/data/wapo/wapo_qrels1"
qrels = ""
for i, row in ds1_wapo.get_qrels().iterrows():
    label = 1 if int(row['label']) > 0 else 0
    line = f"{row['qid']} 0 {row['docno']} {label}\n"
    qrels+= line

with open(wapo_qrel_path, "w") as f:
    f.write(qrels)

In [7]:
pt.Experiment(
    [bm25],
    ds1_wapo.get_topics('title'),
    ds1_wapo.get_qrels(),
    eval_metrics=["map", "recip_rank", "P_10", "ndcg_cut_10"],
    names=["BM25"],
    verbose=True
)

[INFO] [starting] https://trec.nist.gov/data/core/topics2018.txt
[INFO] [finished] https://trec.nist.gov/data/core/topics2018.txt: [00:00] [24.1kB] [59.9MB/s]
[INFO] [starting] https://trec.nist.gov/data/core/qrels2018.txt            
[INFO] [finished] https://trec.nist.gov/data/core/qrels2018.txt: [00:00] [1.12MB] [1.28MB/s]
pt.Experiment: 100%|██████████| 1/1 [00:10<00:00, 10.95s/system]          


,name,map,recip_rank,P_10,ndcg_cut_10
0,BM25,0.168733,0.663436,0.404,0.37107


In [9]:
pt.Experiment(
    [mono_pipeline],
    ds1_wapo.get_topics('title'),
    ds1_wapo.get_qrels(),
    eval_metrics=["map", "recip_rank", "P_10", "ndcg_cut_10"],
    names=["MonoT5"],
    verbose=True
)

pt.Experiment: 100%|██████████| 1/1 [03:42<00:00, 222.32s/system]


,name,map,recip_rank,P_10,ndcg_cut_10
0,MonoT5,0.20841,0.670647,0.476,0.446702


In [ ]:
pt.Experiment(
    [mono_pipeline_500],
    ds1_wapo.get_topics('title'),
    ds1_wapo.get_qrels(),
    eval_metrics=["map", "recip_rank", "P_10", "ndcg_cut_10"],
    names=["MonoT5"],
    verbose=True
)

In [10]:
bm25.search("women in parliament")

,qid,docid,docno,rank,score,query
0,1,563223,f233ecdeb87a44a6aa9ac429999d2d4c,0,19.370159,women in parliament
1,1,486152,4a7c2970fd9bf65fe09c7cf46df7b06d,1,18.690675,women in parliament
2,1,486153,9171debc316e5e2782e0d2404ca7d09d,2,18.690675,women in parliament
3,1,352722,34d443eec1add515a2fbc4af2c8a3a57,3,18.372576,women in parliament
4,1,546574,f1ab493726e1e6f5dd90615d5a1b58b8,4,18.340060,women in parliament
...,...,...,...,...,...,...
195,1,351935,a64ab05765cb2b25a6f18c03b20f5a3a,195,13.074723,women in parliament
196,1,351936,8d0f44ec22604cd5c08d74fc8ffa7cf4,196,13.074723,women in parliament
197,1,351954,56b94f8fd53b63608931a373f813b7b1,197,13.074421,women in parliament
198,1,305768,583e232f566fe972d5d1c1e5652bbb75,198,13.058146,women in parliament


In [9]:
res = mono_pipeline_50.search("women in parliament")

monoT5: 100%|██████████| 1/1 [00:21<00:00, 21.10s/batches]


In [ ]:
pt.Experiment(
    [mono_pipeline_50
    ],
    ds1_wapo.get_topics('title')[:2],
    ds1_wapo.get_qrels(),
    eval_metrics=["map", "recip_rank", "P_10", "ndcg_cut_10"],
    names=["MonoT5"],
    verbose=True
)

In [16]:
import pickle

In [52]:
topic_path = f"workspace/data/wapo/wapo_topics.pkl"
topics = pickle.load(open(topic_path, "rb"))

In [65]:
#generate list of low signal terms from desription/narrative

all_terms = []

for i, row in topics.iterrows():
    all_terms += row['description'].split(" ")
    all_terms += row['narrative'].split(" ")

from collections import Counter

cnt_list = Counter(all_terms)
cnt_list = {k:v for k, v in sorted(cnt_list.items(), key=lambda item: -item[1])}

In [72]:
cnt_list

{'the': 149,
 'of': 107,
 'to': 64,
 'that': 57,
 'in': 56,
 'or': 56,
 'are': 50,
 'documents': 45,
 'and': 45,
 'a': 40,
 'for': 34,
 'relevant.': 33,
 'not': 29,
 'on': 27,
 'relevant': 25,
 'Relevant': 22,
 'A': 21,
 'Find': 21,
 'is': 20,
 'discuss': 20,
 'document': 20,
 'be': 19,
 'as': 19,
 'by': 18,
 'have': 17,
 'describe': 17,
 'will': 15,
 'Documents': 14,
 'any': 12,
 'from': 12,
 'U.S.': 12,
 'What': 12,
 'an': 11,
 'include': 10,
 'countries': 9,
 'would': 9,
 'other': 9,
 'Identify': 9,
 'specific': 9,
 'their': 8,
 'also': 8,
 'efforts': 8,
 'The': 8,
 'being': 8,
 '': 8,
 'which': 7,
 'been': 7,
 'taken': 7,
 'contain': 7,
 'about': 7,
 'mention': 7,
 'such': 7,
 'use': 7,
 'must': 7,
 'either': 7,
 'between': 6,
 'there': 6,
 'food': 6,
 'steps': 6,
 'increase': 6,
 'with': 6,
 'human': 6,
 'people': 6,
 'can': 6,
 'relevant,': 6,
 'treatment': 6,
 'identify': 6,
 'women': 5,
 'has': 5,
 'prevent': 5,
 'species': 5,
 'information': 5,
 'may': 5,
 'cause': 5,
 'reason

In [41]:
terms_description = set(a['description'][0].split(" "))
terms_narrative = set(a['narrative'][0].split(" "))
terms_narrative.union(terms_description)

In [ ]:
terms_narrative.union(terms_description)

In [79]:
# generate user for sim config

user_xmls = sorted(glob.glob("/workspace/sims/users/*"))

In [84]:
for user_xml in user_xmls:
    if 'wapo' in user_xml and 'snip' in user_xml:
        print(f"<user configurationFile=\"{user_xml}\"/>")

<user configurationFile="/workspace/sims/users/gpt+_snipsummarytf2_10rpp_wapo.xml"/>
<user configurationFile="/workspace/sims/users/gpt+_snipsummarytf3_10rpp_wapo.xml"/>
<user configurationFile="/workspace/sims/users/gpt+_snipsummarytf4_10rpp_wapo.xml"/>
<user configurationFile="/workspace/sims/users/gpt+_snipsummarytf5_10rpp_wapo.xml"/>
<user configurationFile="/workspace/sims/users/gpt+_snipsummarytf6_10rpp_wapo.xml"/>


In [86]:
# generate gpt* queries (query variation only over the last term)

gpt_p_lines = open("/workspace/data/nyt/gpt+.txt", "r").read().split("\n")
gpt_p = [line.split(",")[-1] for line in gpt_p_lines]

gpt_p_terms = set(" ".join(gpt_p).split(" "))

In [87]:
from workspace.simiir.query_generators.utils import IdfProvider, get_stopwords, filter_keywords
import random

In [88]:
idf_provider = IdfProvider(index_path="/app/indices/nyt/data.properties", stopwords=get_stopwords())

In [91]:
def generate_gpt_canidate_terms(t_id, gpt_p_lines, idf_provider):
    t_id_terms = []

    for line in gpt_p_lines:
        parts = line.split(",")
        if parts[2] == t_id:
            t_id_terms += parts[3].split(" ")

    t_id_terms = list(set(filter_keywords(idf_provider, t_id_terms)))

    random.shuffle(t_id_terms)
    
    return t_id_terms

In [ ]:
generate_gpt_canidate_terms("321", gpt_p_lines, idf_provider)

In [93]:
seed_queries = open("workspace/data/nyt/title_queries", "r").read().split("\n")

In [94]:
gpt_s_queries = ""

for query in seed_queries:
    gpt_s_queries += query + "\n"
    parts = query.split(",")
    candidate_terms = generate_gpt_canidate_terms(parts[2], gpt_p_lines, idf_provider)
    for i in range(2,101):
        if i >= len(candidate_terms):
            r_index = random.randrange(0, len(candidate_terms))
            line = f"{i},1,{parts[2]},{parts[-1]} {candidate_terms[r_index]}\n"
        else:
            line = f"{i},1,{parts[2]},{parts[-1]} {candidate_terms[i-2]}\n"
        gpt_s_queries += line

In [95]:
with open("workspace/data/nyt/gpt*.txt", "w") as f:
    f.write(gpt_s_queries)

5